In [2]:

import re
import glob
from multiprocessing import Pool
import numpy as np
import pandas as pd
import nltk

In [5]:
# Constants
TITLE = re.compile("(\/)(.*?)(\.)")
HTML_TAG = re.compile("<.*?>")

STOP_WORDS = nltk.corpus.stopwords.words('portuguese')

In [85]:
class Subtitle:
    def __init__(self, filepath):
        self._title = ""
        self._text = []
        self.genre = ""
        self._encoding = ""
        
        self._initialize(filepath)
        
    @property
    def text(self):
        return self._text
    
    @property
    def title(self):
        return self._title
    
    @property
    def encoding(self):
        return self._encoding
    
    def _initialize(self, filepath):
        #TODO: check why chardet is wrongly classifing latin-1 files
#         self._encoding = self._get_encoding(filepath)
        self._encoding = "latin-1"
        self._title = self._parse_title(filepath)
        self._text = self._parse_text(filepath, self._encoding)
    
    def _get_encoding(self, filepath):
        raw_data = open(filepath, 'rb').read()
        return chardet.detect(raw_data)['encoding']
    
    def _parse_title(self, filepath):
        title = TITLE.search(filepath)
        return title.group(2).strip()
    
    def _parse_text(self, filepath, encoding):
        def parse_block(block):
            lines   = block.split('\n')
            txt     = ' '.join(lines[2:])
            txt = self._canonize(txt)
            return txt
        # We don't consider the last three blocks, since usually they're credits for the translators and/or 
        # style definition for the subtitles
        with open(filepath, encoding="latin-1") as f:
            sub_file = f.read()
            sub_file = sub_file.strip().replace('\r', '').split('\n\n')[:-3]
            lines = map(parse_block, sub_file)
            return ' '.join(lines).strip()
    
    def _canonize(self, text):
        text = HTML_TAG.sub("", text)
        return text
    
    def __eq__(self, other):
        return (isinstance(other, self__class__)
               and self._title == other.title
               and self._text == other.text )

In [86]:
def parse_file(filepath):
#     print(filepath)
    return Subtitle(filepath)

def create_subs(files, num_processes=4, chunksize=10):
    # create pool of processes to create Legenda objects
    pool = Pool(processes=num_processes)
    result = pool.map(parse_file, files, chunksize)
    
    return result

In [12]:
files = glob.glob('Legendas/*.srt')
files[0:10]

['Legendas/ Beira do Caminho (2012).DVDRip.3LT0N.br.srt',
 'Legendas/10000 B.C.(2008).XViD-PreVaill.br.srt',
 'Legendas/12 Rounds.DVDRip.aXXo.br.srt',
 'Legendas/127 Hours (2010).BDRip.Larceny.br.srt',
 'Legendas/15 Minutes(2001).br.srt',
 'Legendas/17 Again.720p.REFiNED.br.srt',
 'Legendas/1984 (1956).br.srt',
 'Legendas/2 Fast 2 Furious.DVDScr.DVL.br.srt',
 'Legendas/20000 Leagues Under The Sea (1954).br.srt',
 'Legendas/21 21 The Movie .R5.PUKKA.br.srt']

In [37]:
%time subs = create_subs(files)

CPU times: user 87.2 ms, sys: 80.9 ms, total: 168 ms
Wall time: 1.29 s


In [41]:
titles = [s.title for s in subs]
subs = [s.text for s in subs]

In [39]:
def tokenize(text, stem=False):
    sent_tokenizer = nltk.data.load('tokenizers/punkt/portuguese.pickle')
    stemmer = nltk.stem.RSLPStemmer()
    
    tokens = [word.lower() for sentence in sent_tokenizer.tokenize(text) \
              for word in nltk.word_tokenize(sentence) if word.isalpha()]
    if stem:
        stems = [stemmer.stem(token) for token in tokens]
        return stems
    else:
        return tokens

## TF-IDF

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

# define TF-IDF parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                   min_df=0.2, stop_words=STOP_WORDS,
                                  use_idf=True, tokenizer=tokenize, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(subs)

print(tfidf_matrix.shape)

CPU times: user 2min 39s, sys: 2.84 s, total: 2min 42s
Wall time: 2min 44s
(631, 1659)


In [58]:
# list of TF-IDF Matrix features (vocabulary of our corpus)
terms = tfidf_vectorizer.get_feature_names()

## K-means Clustering

In [61]:
from sklearn.cluster import KMeans

# TODO: use method from http://dl.acm.org/citation.cfm?doid=99935.99938
# using the number of movies genres
num_clusters = 12
kmean = KMeans(n_clusters=num_clusters, n_jobs=-1)

%time kmean.fit(tfidf_matrix)

CPU times: user 1.23 s, sys: 1.37 s, total: 2.59 s
Wall time: 4.02 s


KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=12, n_init=10,
    n_jobs=-1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

## K - Nearest Neighbors

In [115]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(algorithm='auto')

%time knn.fit(tfidf_matrix)

TypeError: fit() missing 1 required positional argument: 'y'

### Some queries

In [96]:
# Queries in the format (title, Subtitle)
queries = {"Ace Ventura Pet Detective": Subtitle("Legendas/Ace Ventura Pet Detective.DVDRip.BugBunny.br.srt"), 
          "Poltergeist": Subtitle("Legendas/Poltergeist(1982).br.srt"),
          "Lord of the Rings The Fellowship of the Ring": Subtitle("Legendas/Lord of the Rings The Fellowship of the Ring The.DVDRip.SecretMyth.br.srt"),
          "Fantastic Four": Subtitle("Legendas/Fantastic Four.DVDRip.br.srt"),
          "Frozen": Subtitle("Legendas/Frozen.720p.BlueRay.YIFY.br.srt")}



In [114]:
from sklearn.metrics.pairwise import linear_kernel

for title, sub in queries.items():
    doc_tfidf = tfidf_vectorizer.transform([sub.text])
    # get the top 6 (since we used all the movies to train, the first one is the doc itself)
    cos_distance = 1 - linear_kernel(doc_tfidf, tfidf_matrix).flatten()
    # we ignore the first argument since is the doc itself (we trained using all movies)
    rank = cos_distance.argsort()[1:6]
    # print rank
    print(title)
    for i, rank_idx in enumerate(rank):
        print("\t{pos} - {title} ({cos_dist:.3})".format(pos=i+1,
                                                     title=titles[int(rank_idx)],
                                                     cos_dist=cos_distance[rank_idx]))
    

Ace Ventura Pet Detective
	1 - Ace Ventura Jr Pet Detective (0.531)
	2 - Jurassic World (0.612)
	3 - Devil s Advocate The (0.626)
	4 - Ace Ventura When Nature Calls (0.627)
	5 - Zodiac (0.647)
Lord of the Rings The Fellowship of the Ring
	1 - Lord of the Rings The Two Towers The (0.385)
	2 - Lord of the Rings The Return of the King The (0.41)
	3 - Ben Hur (1959) (0.492)
	4 - The Hobbit An Unexpected Journey (2012) (0.502)
	5 - Mis rables Les (2013) (0.546)
Fantastic Four
	1 - Imposter The (2008) (0.336)
	2 - Ghost Rider (0.404)
	3 - Intouchables Untouchable  (2011) (0.404)
	4 - Time Traveler s Wife The (0.409)
	5 - Cinderella Story A (0.411)
Poltergeist
	1 - Volver (0.53)
	2 - Others The (2001) (0.535)
	3 - Premonition(2007) (0.567)
	4 - Paranormal Activity The Ghost Dimension (0.593)
	5 - Shrek 2 (2004) (0.603)
Frozen
	1 - Shrek (0.403)
	2 - Blonde and Blonder(2007) (0.53)
	3 - Butterfly Effect 2 The (0.537)
	4 - A Life Less Ordinary (0.54)
	5 - Open Season (0.55)


### Relatory

   #### Ace Ventura Pet Detective:
   O primeiro filme mais proximo (Ace Ventura Jr Pet Detective) faz todo sentido ser o mais proximo, já que o mesmo vêm é um filme da mesma franquia. Porêm algo interessante é o fato dos filmes _Jurassic World_ e _The Devil's Advocata_ terem uma maior semelhança que outro filme da mesma franquia. Outro resultado estranho é o fato de o filme Zodiac, que é um filme sobre um Serial Killer está fortemente relacionado a esse filme.
   
   #### Lord of the Rings The Fellowship of the Ring
   Os dois primeiros resultados são o esperado, dado que são os outros filmes da mesma trilogia. O filme Ben Hur é um filme que conta a historia de um guerreiro, portanto a linguagem utilizada provavelmente se assemelha ao estilo utilizado na terra média, porêm sua posição me é interessante por ser mais proximo que outro filme do mesmo universo (_The Hobbit An Unexpected Journey_). O filme Les Miserables, eu não consigo entender bem porque ele é considerado proximo do Senhor dos Aneis.
   
   #### Fantastic Four
   É um dos mais interessantes tento em vista que a sequência do mesmo não está presente no top 5. Na verdade o unico filme to top-5 que faz algum sentido é o _Ghost Rider_. Um fato bastante curioso é o filme _The Imposter_ que conta a historia de um _Con Artist_ que engana uma familia para pensar que o mesmo é um mebro da mesma que desapareceu, eu não consigo encontrar nenhuma conexão com esse filme e o quarteto fantastico; tambem não consigo encontrar muita conexão entre os outros filmes no top-5 (_Intouchables Untouchable_, _The Time Traveler's Wife_ e _Cinderella_)
   
   #### Poltergeist
   Todos os filmes no top-5 (a parte de Shrek 2) fazem sentido pois são filmes de fantasmas e atividades paranormais. O filme Shrek é bastante curioso para mim, já que o mesmo é um filme infantil, com pouca ou nenhuma ligação a um filme de terror.
   
   #### Frozen
   O primeiro filme _Shrek_ faz todo sentido já que conta a historia de uma princesa com uma maldição, o que é semelhante ao nucleo da historio de _Frozen_. Retirando _Open Season_ que é um filme infantil, porêm com pouca semelhança com o plot de Frozen, todos os outros filmes não fazem muito sentido, _Blonde and Blonder_ é um besteirol americano, _The Butterfly Effect 2_ é um filme de ficção ciêntifica e o filme _A Life Less Ordinary_ é um romance.